# DDPMs

https://colab.research.google.com/github/huggingface/notebooks/blob/main/examples/annotated_diffusion.ipynb#scrollTo=6fe49a34

 - Made of a forward process and a reverse process
 - Both are indexed by a discrete timestep $t$ for a finite number of timesteps

## Forward Process - $q$

 - $q$ gradually adds Gaussian noise to an image until you end up with pure noise
 - Starting at $t=0$, where you sample a real image from the dataset - $\textbf{x}_0 \sim q(\textbf{x}_0)$
 - At each stage, $q$ samples noise from a Gaussian distribution and adds it to the image from the previous timestep - $q(\textbf{x}_t | \textbf{x}_{t-1})$ with variance schedule $(0 < \beta_1 < \beta_2 < ... < \beta_T < 1)$
 - $ q(\mathbf{x}_t | \mathbf{x}_{t-1}) = \mathcal{N}(\mathbf{x}_t; \sqrt{1 - \beta_t} \mathbf{x}_{t-1}, \beta_t \mathbf{I}) $
 - Each new, noisier image is drawn from a conditional Gaussian with $\mathbf{\mu}_t = \sqrt{1 - \beta_t} \mathbf{x}_{t-1}$ and $\sigma^2_t = \beta_t$
   - We sample $\mathbf{\epsilon} \sim \mathcal{N}(\mathbf{0}, \mathbf{I})$.
   - We set $\mathbf{x}_t = \sqrt{1 - \beta_t} \mathbf{x}_{t-1} +  \sqrt{\beta_t} \mathbf{\epsilon}$.
 - Given a sufficiently large $T$ and a well-behaved noise schedule, you end up with an isotropic Gaussian at $t=T$
 - We can sample $\mathbf{x}_t$ at any arbitrary noise level conditioned on $mathbf{x}_0$ (since sums of Gaussians is also Gaussian
   - We don't need to apply $q$ repeatedly in order to sample $\mathbf{x}_t$
   - We have that $q(\mathbf{x}_t | \mathbf{x}_0) = \cal{N}(\mathbf{x}_t; \sqrt{\bar{\alpha}_t} \mathbf{x}_0, (1- \bar{\alpha}_t) \mathbf{I})$
   - $\alpha_t := 1 - \beta_t$
   - $\bar{\alpha}_t := \Pi_{s=1}^{t} \alpha_s$
 - $\bar{\alpha}_t$ are functions of the known $\beta_t$ variance schedule and thus are also known and can be precomputed

## Reverse Process - $p$

 - $p$ reverses the added Gaussian noise
 - If we knew the conditional distribution $p(\mathbf{x}_{t-1} | \mathbf{x}_t)$, then we could reverse the process by sampling some random noise as $\textbf{x}_T$ and gradually denoise it till we reach $\textbf{x}_0$
 - We do not know $p(\mathbf{x}_{t-1} | \mathbf{x}_t)$ - it is intractible as it requires knowing the distribution of all possible images
 - Instead, we leverage a neural network to learn the conditional probability $p_\theta(\mathbf{x}_{t-1} | \mathbf{x}_t)$, where $\theta$ are the parameters of the neural network
 - We assume the reverse process is also Gaussian, so it is just parameterised by $\mu_\theta$ and $\Sigma_\theta$
 - Therefore, $p_\theta (\mathbf{x}_{t-1} | \mathbf{x}_t) = \mathcal{N}(\mathbf{x}_{t-1}; \mu_\theta(\mathbf{x}_{t},t), \Sigma_\theta (\mathbf{x}_{t},t))$

### The Neural Network
 - Our neural network needs to learn mean and variance, but [2020] keeps the variance fixed and just learns the mean
 - We set $\Sigma_\theta ( \mathbf{x}_t, t) = \sigma^2_t \mathbf{I}$ to untrained time constants
 - An objective function is a function that is maximised or minimised
 - The combination of $q$ and $p_0$ can be seen as a VAE, so we can minimise the ELBO to minimise the NLL with respect to the ground truth $\textbf{x}_0$
 - The ELBO is $L = L_0 + \dots + L_T$
 - Each term $L_t$, except $L_0$ is a KL-divergence between two Gaussians, so can be written as an L2 loss wrt. the means
 - As we can arbitrarily sample $x_t$, during training, we can optimize random terms of the loss function $L$
   - i.e., we can randomly sample $t$ during training and optimize $L_t$